In [39]:
from subtitle_process import fetch_subtitle
import json
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import getpass
import os
import nest_asyncio
import asyncio
import re
from typing import Any, Dict

nest_asyncio.apply()

In [40]:
#从页面所有BV号构建url
urls = []
with open('url.json', 'r') as file:
    urls = json.load(file)

urls = ['https://'+item+'/' for item in urls]

In [41]:
#将前25个视频的字幕加入subtitles
subtitles = []
for url in urls[:5]:
    sub = fetch_subtitle(url)
    subtitles.append(sub)

len(subtitles)

Extracted segment: 13S411A73f
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/19058691521601443768adce9ddfec737839346a06b339dd1f2a?auth_key=1720064088-44b67f1a4eb1402e99cb1f2f95aff7a7-0-93cd2e561120953e5cea4f0cb77c8593


KeyboardInterrupt: 

In [ ]:
len(subtitles)

In [ ]:
template_sub = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一些相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

### 相似题二：
（构造的相似题目二）

### 相似题二讲解：
（构造的相似题目二的讲解）

视频字幕：
{subtitle}

注意：对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话。
"""

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [ ]:
'''
输入一：一个包含template中所有变量以及对应值的字典
输入二：一个给gpt的prompt template
输入三：回答token数量限制, 默认1000
输出：gpt的回答
'''
async def inference(params: Dict[str, any], template: str, max_tokens=1000) -> str:
    
    output = ""
    llm = ChatOpenAI(
        model='gpt-4o',
        temperature=0,
        max_tokens=max_tokens,
        max_retries=2,
        timeout=20,
    )

    prompt = PromptTemplate.from_template(template=template)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    async for chunk in chain.astream(params, version="v2"):
        output += chunk
        print(chunk, end='', flush=True)

    return output

In [48]:
a = await inference({"subtitle" : subtitles[0]}, template_sub, 1000)
a

IndexError: list index out of range

In [47]:
responses = []
for subtitle in subtitles:
    response = await inference({"subtitle": subtitle}, template_sub, 1000)
    responses.append(response)

extract_question_explanations(responses)

In [46]:

def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)\n\n### 相似题二：\n(.+?)\n\n### 相似题二讲解：\n(.+?)\n", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
                "similar_question_2": match[4].strip(),
                "similar_explanation_2": match[5].strip(),
            })

    with open('questions_explanation.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)